# **Création de la Database Argent**

## **Imports et Dataframes**

### **Imports et création de la database**

In [110]:
import pandas as pd
import datetime as dt

from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base

import os

In [111]:
try:
    os.remove("db_argent.sqlite")
    print('Base de données argent supprimée.')
except:
    print('Pas de base de données argent.')

Pas de base de données argent.


In [112]:
base = declarative_base()

engine = create_engine("sqlite:///db_bronze.sqlite", echo=True)
engine_argent = create_engine("sqlite:///db_argent.sqlite", echo=False)

/tmp/ipykernel_12709/3632543521.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  base = declarative_base()


In [113]:
from sqlalchemy import text

list_data = []

with engine.connect() as connection:
    result = connection.execute(text("""
    select *

    from 'Order_items' as Items
    join 'Order_payments' as Payments on Items.order_id=Payments.order_id 
    join 'Sellers_dataset' as Sellers on Items.seller_id=Sellers.seller_id 
    join 'Order_reviews' as Reviews on Items.order_id=Reviews.order_id inner  
    join 'Products_dataset' as Products on Products.product_id=Items.product_id 
    join 'Category_names' as Categorys on Categorys.product_category_name=Products.product_category_name 
    join 'Order_dataset' as Orders on Items.order_id=Orders.order_id 
    join 'Customers' as Customers on Customers.customer_id=Orders.customer_id
    """))
    
    for row in result:
        list_data.append(row)

data = pd.DataFrame(list_data)

2023-02-14 16:36:02,179 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-14 16:36:02,179 INFO sqlalchemy.engine.Engine 
    select *

    from 'Order_items' as Items
    join 'Order_payments' as Payments on Items.order_id=Payments.order_id 
    join 'Sellers_dataset' as Sellers on Items.seller_id=Sellers.seller_id 
    join 'Order_reviews' as Reviews on Items.order_id=Reviews.order_id inner  
    join 'Products_dataset' as Products on Products.product_id=Items.product_id 
    join 'Category_names' as Categorys on Categorys.product_category_name=Products.product_category_name 
    join 'Order_dataset' as Orders on Items.order_id=Orders.order_id 
    join 'Customers' as Customers on Customers.customer_id=Orders.customer_id
    
2023-02-14 16:36:02,181 INFO sqlalchemy.engine.Engine [generated in 0.00209s] ()
2023-02-14 16:36:07,480 INFO sqlalchemy.engine.Engine ROLLBACK


### **Création du Dataframe**

In [114]:
df = data.copy()
df.drop(['index'], axis=1, inplace=True)

In [115]:
pd.options.display.max_columns = None

In [116]:
range_col = range(len(df.columns))
name_col = df.columns

list_col_name = []
list_col_num = []

for index_col, col in zip(range_col, name_col):
    if col not in list_col_name:
        list_col_name.append(col)
        list_col_num.append(index_col)

In [117]:
df = df.iloc[:, list_col_num]
df.drop(['product_category_name'], axis=1, inplace=True)

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,None,"Perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,None,None,2017-05-13 00:00:00,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,None,Chegou antes do prazo previsto e o produto sur...,2018-01-23 00:00:00,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,None,None,2018-08-15 00:00:00,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,1,credit_card,3,218.04,87900,loanda,PR,940144190dcba6351888cafa43f3a3a5,None,Gostei pois veio no prazo determinado .,2017-03-02 00:00:00,2017-03-03 10:54:59,5,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,None,None,2018-05-11 00:00:00,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,housewares,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00,

In [118]:
df = df.drop_duplicates()

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,None,"Perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,None,None,2017-05-13 00:00:00,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,None,Chegou antes do prazo previsto e o produto sur...,2018-01-23 00:00:00,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,None,None,2018-08-15 00:00:00,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,1,credit_card,3,218.04,87900,loanda,PR,940144190dcba6351888cafa43f3a3a5,None,Gostei pois veio no prazo determinado .,2017-03-02 00:00:00,2017-03-03 10:54:59,5,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,garden_tools,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,None,None,2018-05-11 00:00:00,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,housewares,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00,

### **Sélection des colonnes**

In [119]:
df['order_number'] = 1

In [120]:
df = df[df.payment_type != 'undefined']
df.payment_sequential = df.payment_sequential.apply(lambda x : 1 if x==1 else 0)
df = df[df.payment_installments != 0]

In [121]:
df.review_comment_message = df.review_comment_message.apply(lambda s : 0 if str(s) == 'None' else len(s))
df.review_comment_title = df.review_comment_title.apply(lambda s : 0 if str(s)=='None' else len(s))
df["nb_char_comment"] = df[['review_comment_message','review_comment_title']].apply(lambda x: x[0]+x[1], axis=1)
df = df.drop(['review_comment_title','review_comment_message'],axis=1)

### **Création des colonnes supplémentaires**

In [122]:
price_total = pd.DataFrame(df.groupby(["customer_unique_id"])['price'].sum()).reset_index()
price_total.rename({'price': 'price_total'}, axis=1, inplace=True)

total_order = pd.DataFrame(df.groupby(["customer_unique_id"])['order_number'].sum()).reset_index()
total_order.rename({'order_number': 'total_order'}, axis=1, inplace=True)

df = df.merge(price_total, how = 'inner' , on = ["customer_unique_id"])
df = df.merge(total_order, how = 'inner' , on = ["customer_unique_id"])

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_number,nb_char_comment,price_total,total_order
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,2017-09-21 00:00:00,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,cool_stuff,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1,46,58.90,1
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,2017-05-13 00:00:00,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,0,252.78,2
2,c105be01f21c9df6a2579f59b95cbcd9,1,d2f5484cbffe4ca766301b21ab9246dd,36a968b544695394e4e9d7572688598f,2017-09-22 16:27:55,12.88,11.85,1,credit_card,1,24.73,11010,santos,SP,c242eb53a2ddb13822240752d36aa9e8,2017-09-26 00:00:00,2017-09-29 11:42:53,5,51.0,448.0,1.0,180.0,16.0,9.0,16.0,computers_accessories,02288a47f696f3054b9ef1d5f8e73973,delivered,2017-09-18 16:12:04,2017-09-18 16:27:55,2017-09-20 20:00:24,2017-09-25 21:59:46,2017-10-04 00:00:00,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,3,252.78,2
3,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,2018-01-23 00:00:00,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,furniture_decor,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1,90,199.00,1
4,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,2018-08-15 00:00:00,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,perfumery,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1,0,12.99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115601,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,2018-05-11 00:00:00,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,housewares,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18 00:00:00,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,1,0,299.99,1
115602,fffcd46ef2263f404302a634eb

In [123]:
df['review_creation_date'] = pd.to_datetime(df['review_creation_date'])
df['review_answer_timestamp'] = pd.to_datetime(df['review_answer_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_carrier_date'] = pd.to_datetime(df['order_delivered_carrier_date'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])
df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'])
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

#### **Super Catégories**

In [124]:
super_categories = {
 'health_beauty': 'Health_Beauty',
 'perfumery': 'Health_Beauty',
 'computers': 'Technology',
 'pc_gamer' : 'Technology',
 'computers_accessories': 'Technology',
 'electronics': 'Technology',
 'tablets_printing_image': 'Technology',
 'telephony': 'Technology',
 'fixed_telephony': 'Technology',
 'auto': 'Automotive',
 'small_appliances_home_oven_and_coffee': 'Home',
 'home_comfort_2': 'Home',
 'home_appliances_2': 'Home',
 'home_confort': 'Home',
 'air_conditioning': 'Home',
 'small_appliances': 'Home',
 'bed_bath_table': 'Home',
 'furniture_decor': 'Home',
 'housewares': 'Home',
 'home_appliances': 'Home',
 'home_comfort': 'Home',
 'kitchen_dining_laundry_garden_furniture': 'Home',
 'portable_kitchen_food_preparers' : 'Home',
 'fashio_female_clothing': 'Fashion_Accessories',
 'luggage_accessories': 'Fashion_Accessories',
 'fashion_bags_accessories': 'Fashion_Accessories',
 'fashion_shoes': 'Fashion_Accessories',
 'watches_gifts': 'Fashion_Accessories',
 'fashion_male_clothing': 'Fashion_Accessories',
 'fashion_female_clothing': 'Fashion_Accessories',
 'fashion_childrens_clothes': 'Fashion_Accessories',
 'fashion_underwear_beach': 'Fashion_Accessories',
 'fashion_sport': 'Fashion_Accessories',
 'sports_leisure': 'Sports_Leisure',
 'food_drink': 'Food_Beverage',
 'la_cuisine': 'Food_Beverage',
 'food': 'Food_Beverage',
 'drinks': 'Food_Beverage',
'baby': 'Baby',
 'diapers_and_hygiene': 'Baby',
 'stationery': 'Office_Supplies',
 'office_furniture': 'Office_Supplies',
 'flowers': 'Gardening',
 'costruction_tools_garden': 'Gardening',
 'books_general_interest': 'Books',
 'books_technical': 'Books',
 'books_imported': 'Books',
 'cds_dvds_musicals': 'Books',
 'dvds_blu_ray': 'Books',
 'art': 'Arts_Music',
 'audio': 'Arts_Music',
 'music': 'Arts_Music',
 'arts_and_craftmanship': 'Arts_Music',
 'cine_photo': 'Arts_Music',
 'musical_instruments': 'Arts_Music',
 'toys': 'Toys_Games',
 'consoles_games': 'Toys_Games',
 'cool_stuff': 'Toys_Games',
 'party_supplies': 'Party_Supplies',
 'christmas_supplies': 'Party_Supplies',
 'home_construction': 'Construction_Tools',
 'costruction_tools_tools': 'Construction_Tools',
 'construction_tools_construction': 'Construction_Tools',
 'garden_tools': 'Construction_Tools',
 'construction_tools_lights': 'Construction_Tools',
 'construction_tools_safety': 'Construction_Tools',
 'pet_shop': 'Pet_Supplies',
 'industry_commerce_and_business': 'Agro_Industry_Commerce',
 'agro_industry_and_commerce': 'Agro_Industry_Commerce',
 'furniture_mattress_and_upholstery': 'Furniture',
 'furniture_living_room': 'Furniture',
 'furniture_bedroom': 'Furniture',
 'market_place': 'Market_place',
 'signaling_and_security': 'Signaling_Security',
 'security_and_services': 'Signaling_Security'
 }

In [125]:
df['product_category_name_english'] = df['product_category_name_english'].replace(super_categories)

#### **Recence**

In [126]:
recence = pd.DataFrame(df.groupby(["customer_unique_id"])['order_purchase_timestamp'].max()).reset_index()
recence.rename({'order_purchase_timestamp': 'recence'}, axis=1, inplace=True)

df = df.merge(recence, how = 'inner' , on = ["customer_unique_id"])

In [127]:
date_max = df['order_purchase_timestamp'].max() + dt.timedelta(days=1)

def recence(date):
    if date_max - date < dt.timedelta(days=91):
        return 15
    elif date_max - date < dt.timedelta(days=182):
        return 10
    elif date_max - date < dt.timedelta(days=273):
        return 6
    elif date_max - date < dt.timedelta(days=365):
        return 3
    else :
        return 0

In [128]:
df['recence'] = df['recence'].apply(lambda x : recence(x))

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_number,nb_char_comment,price_total,total_order,recence
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,2017-09-21,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,Toys_Games,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1,46,58.90,1,3
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,2017-05-13,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,Pet_Supplies,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,0,252.78,2,3
2,c105be01f21c9df6a2579f59b95cbcd9,1,d2f5484cbffe4ca766301b21ab9246dd,36a968b544695394e4e9d7572688598f,2017-09-22 16:27:55,12.88,11.85,1,credit_card,1,24.73,11010,santos,SP,c242eb53a2ddb13822240752d36aa9e8,2017-09-26,2017-09-29 11:42:53,5,51.0,448.0,1.0,180.0,16.0,9.0,16.0,Technology,02288a47f696f3054b9ef1d5f8e73973,delivered,2017-09-18 16:12:04,2017-09-18 16:27:55,2017-09-20 20:00:24,2017-09-25 21:59:46,2017-10-04,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,3,252.78,2,3
3,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,2018-01-23,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,Home,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1,90,199.00,1,6
4,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,2018-08-15,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,Health_Beauty,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1,0,12.99,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115601,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,2018-05-11,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,Home,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,1,0,299.99,1,10
115602,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,

#### **Frequence**

In [129]:
df_freq = df[df['order_purchase_timestamp'] >= (date_max - dt.timedelta(days=365))]

df_freq

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_number,nb_char_comment,price_total,total_order,recence
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,2017-09-21,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,Toys_Games,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1,46,58.90,1,3
2,c105be01f21c9df6a2579f59b95cbcd9,1,d2f5484cbffe4ca766301b21ab9246dd,36a968b544695394e4e9d7572688598f,2017-09-22 16:27:55,12.88,11.85,1,credit_card,1,24.73,11010,santos,SP,c242eb53a2ddb13822240752d36aa9e8,2017-09-26,2017-09-29 11:42:53,5,51.0,448.0,1.0,180.0,16.0,9.0,16.0,Technology,02288a47f696f3054b9ef1d5f8e73973,delivered,2017-09-18 16:12:04,2017-09-18 16:27:55,2017-09-20 20:00:24,2017-09-25 21:59:46,2017-10-04,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,3,252.78,2,3
3,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,2018-01-23,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,Home,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1,90,199.00,1,6
4,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,2018-08-15,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,Health_Beauty,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1,0,12.99,1,15
7,00054e8431b9d7675808bcb819fb4a32,1,8d4f2bb7e93e6710a28f34fa83ee7d28,7040e82f899a04d1b434b795a43b4617,2017-12-14 12:10:31,19.90,11.85,1,credit_card,1,31.75,1026,sao paulo,SP,0381de7572d99d75230ce91207807265,2017-12-19,2017-12-29 02:40:12,4,52.0,815.0,1.0,200.0,27.0,5.0,20.0,Technology,32e2e6ab09e778d99bf2e0ecd4898718,delivered,2017-12-10 11:53:48,2017-12-10 12:10:31,2017-12-12 01:07:48,2017-12-18 22:03:38,2018-01-04,635d9ac1680f03288e72ada3a1035803,16700,guararapes,SP,1,0,19.90,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115600,fffbee3b5462987e66fb49b1c5411df2,1,6f0169f259bb0ff432bfff7d829b9946,213b25e6f54661939f11710a6fddb871,2018-06-28 09:58:03,119.85,20.03,1,credit_card,3,139.88,13321,salto,SP,a614fd24fbd8348db190dc720530e101,2018-07-06,2018-07-09 10:41:17,5,58.0,918.0,6.0,1050.0,16.0,20.0,16.0,Construction_Tools,11a0e041ea6e7e21856d2689b64e7f3a,delivered,2018-06-19 09:27:48,2018-06-19 09:58:03,2018-06-29 13:46:00,2018-07-05 17:51:08,2018-07-23,ecc3d4eb9b17d2f0865d21f2abecc51c,39401,montes claros,MG,1,0,119.85,1,15
115601,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01

In [130]:
total_order_year = pd.DataFrame(df_freq.groupby(["customer_unique_id"])['order_number'].sum()).reset_index()
total_order_year.rename({'order_number': 'frequence'}, axis=1, inplace=True)

df = df.merge(total_order_year, how = 'outer' , on = ["customer_unique_id"])

df['frequence'] = df['frequence'].apply(lambda x : x * 3 if x >= 0 else 0)

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_number,nb_char_comment,price_total,total_order,recence,frequence
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,2017-09-21,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,Toys_Games,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1,46,58.90,1,3,3.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,2017-05-13,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,Pet_Supplies,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,0,252.78,2,3,3.0
2,c105be01f21c9df6a2579f59b95cbcd9,1,d2f5484cbffe4ca766301b21ab9246dd,36a968b544695394e4e9d7572688598f,2017-09-22 16:27:55,12.88,11.85,1,credit_card,1,24.73,11010,santos,SP,c242eb53a2ddb13822240752d36aa9e8,2017-09-26,2017-09-29 11:42:53,5,51.0,448.0,1.0,180.0,16.0,9.0,16.0,Technology,02288a47f696f3054b9ef1d5f8e73973,delivered,2017-09-18 16:12:04,2017-09-18 16:27:55,2017-09-20 20:00:24,2017-09-25 21:59:46,2017-10-04,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,3,252.78,2,3,3.0
3,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,2018-01-23,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,Home,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1,90,199.00,1,6,3.0
4,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,2018-08-15,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,Health_Beauty,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1,0,12.99,1,15,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115601,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,2018-05-11,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,Home,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,1,0,299.99,1,10,3.0
115602,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb

#### **Montant**

In [131]:
def montant (prix):
    
    price = round(prix * 2 / 100)

    if price >= 10:
        price = 10
    
    return price

In [132]:
df['montant'] = 0
df['montant'] = df['price'].apply(lambda x : montant(x))

df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,seller_zip_code_prefix,seller_city,seller_state,review_id,review_creation_date,review_answer_timestamp,review_score,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_number,nb_char_comment,price_total,total_order,recence,frequence,montant
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,27277,volta redonda,SP,97ca439bc427b48bc1cd7177abe71365,2017-09-21,2017-09-22 10:57:03,5,58.0,598.0,4.0,650.0,28.0,9.0,14.0,Toys_Games,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,1,46,58.90,1,3,3.0,1
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,3471,sao paulo,SP,7b07bacd811c4117b742569b04ce3580,2017-05-13,2017-05-15 11:34:13,4,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,Pet_Supplies,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,0,252.78,2,3,3.0,5
2,c105be01f21c9df6a2579f59b95cbcd9,1,d2f5484cbffe4ca766301b21ab9246dd,36a968b544695394e4e9d7572688598f,2017-09-22 16:27:55,12.88,11.85,1,credit_card,1,24.73,11010,santos,SP,c242eb53a2ddb13822240752d36aa9e8,2017-09-26,2017-09-29 11:42:53,5,51.0,448.0,1.0,180.0,16.0,9.0,16.0,Technology,02288a47f696f3054b9ef1d5f8e73973,delivered,2017-09-18 16:12:04,2017-09-18 16:27:55,2017-09-20 20:00:24,2017-09-25 21:59:46,2017-10-04,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,1,3,252.78,2,3,3.0,0
3,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,37564,borda da mata,MG,0c5b33dea94867d1ac402749e5438e8b,2018-01-23,2018-01-23 16:06:31,5,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,Home,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,1,90,199.00,1,6,3.0,4
4,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,14403,franca,SP,f4028d019cb58564807486a6aaf33817,2018-08-15,2018-08-15 16:39:01,4,42.0,480.0,1.0,200.0,16.0,10.0,15.0,Health_Beauty,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,1,0,12.99,1,15,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115601,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,1,boleto,1,343.40,88303,itajai,SC,9185f849f32d82e216a4e025e0c50f5c,2018-05-11,2018-05-14 12:53:47,5,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,Home,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,1,0,299.99,1,10,3.0,6
115602,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c82

#### **RFM**

In [133]:
df['rfm'] = df[['recence', 'frequence', 'montant']].apply(lambda x : x[0] + x[1] + x[2], axis=1)
del df['recence']
del df['frequence']
del df['montant']

#### **Convert time to numeric**

In [136]:
df['response_time'] = df['review_answer_timestamp'] - df['review_creation_date']

In [137]:
df['review_creation_date'] = pd.to_numeric(df['review_creation_date'])
df['review_answer_timestamp'] = pd.to_numeric(df['review_answer_timestamp'])
df['order_approved_at'] = pd.to_numeric(df['order_approved_at'])
df['order_delivered_carrier_date'] = pd.to_numeric(df['order_delivered_carrier_date'])
df['order_delivered_customer_date'] = pd.to_numeric(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_numeric(df['order_estimated_delivery_date'])
df['shipping_limit_date'] = pd.to_numeric(df['shipping_limit_date'])
df['response_time'] = pd.to_numeric(df['response_time'])
df['order_purchase_timestamp'] = pd.to_numeric(df['order_purchase_timestamp'])

In [138]:
df.drop(['order_id', 'order_item_id', 'product_id', 'seller_id', 'review_id', 'customer_id', 'order_number', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1, inplace=True)

df

,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_creation_date,review_answer_timestamp,review_score,product_category_name_english,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,nb_char_comment,price_total,total_order,rfm,response_time
0,1505814335000000000,58.90,13.29,1,credit_card,2,72.19,1505952000000000000,1506077823000000000,5,Toys_Games,delivered,1505293142000000000,1505295935000000000,1505846056000000000,1505951028000000000,1506643200000000000,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,46,58.90,1,7.0,125823000000000
1,1493809513000000000,239.90,19.93,1,credit_card,3,259.83,1494633600000000000,1494848053000000000,4,Pet_Supplies,delivered,1493203986000000000,1493204713000000000,1493908500000000000,1494605064000000000,1494806400000000000,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,0,252.78,2,11.0,214453000000000
2,1506097675000000000,12.88,11.85,1,credit_card,1,24.73,1506384000000000000,1506685373000000000,5,Technology,delivered,1505751124000000000,1505752075000000000,1505937624000000000,1506376786000000000,1507075200000000000,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,3,252.78,2,6.0,301373000000000
3,1516286910000000000,199.00,17.87,1,credit_card,5,216.87,1516665600000000000,1516723591000000000,5,Home,delivered,1515940411000000000,1515941310000000000,1516106208000000000,1516627156000000000,1517788800000000000,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,90,199.00,1,13.0,57991000000000
4,1534327818000000000,12.99,12.79,1,credit_card,2,25.78,1534291200000000000,1534351141000000000,4,Health_Beauty,delivered,1533722435000000000,1533723018000000000,1533907680000000000,1534253559000000000,1534723200000000000,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,0,12.99,1,18.0,59941000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115601,1525234261000000000,299.99,43.41,1,boleto,1,343.40,1525996800000000000,1526302427000000000,5,Home,delivered,1524491826000000000,1524629461000000000,1524658140000000000,1525993000000000000,1526601600000000000,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,0,299.99,1,19.0,305627000000000
115602,1532061108000000000,350.00,36.53,1,boleto,1,386.53,1532390400000000000,1532510729000000000,5,Technology,delivered,1531564006000000000,1531801908000000000,1531814700000000000,1532377915000000000,1533081600000000000,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR,0,350.00,1,25.0,120329000000000
115603,1509383665000000000,99.90,16.95,1,credit_card,3,116.85,1509235200000000000,1509312832000000000,5,Sports_Leisure,delivered,1508778476000000000,1508865265000000000,1509030794000000000,1509193342000000000,1510272000000000000,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP,0,99.90,1,8.0,77632000000000
115604,1503273872000000000,55.99,8.72,1,credit_card,3,64.71,1502928000000000000,1503059045000000000,5,Technology,delivered,1502751779000000000,1502755472000000000,1502823773000000000,1502920780000000000,1503619200000000000,eb803377c9315b564bdedad672039306,13289,vinhedo,SP,47,55.99,1,1.0,131045000000000


In [139]:
pd.to_datetime(df['shipping_limit_date'][0])

Timestamp('2017-09-19 09:45:35')

### **Aggregate**

In [140]:
last_col = ['shipping_limit_date', 'review_creation_date', 'review_answer_timestamp', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'response_time']
mean_col = ['price', 'freight_value', 'payment_value', 'payment_installments', 'review_score', 'price_total', 'total_order', 'rfm']
mode_col = ['payment_sequential', 'payment_type', 'product_category_name_english', 'order_status', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
sum_col = ['nb_char_comment']

dict_agg = {}

for col in mode_col:
    dict_agg[f'{col}'] = pd.Series.mode

for col in last_col:
    dict_agg[f'{col}'] = 'last'

for col in mean_col:
    dict_agg[f'{col}'] = 'mean'

for col in sum_col:
    dict_agg[f'{col}'] = 'sum'

In [141]:
dict_agg

{'payment_sequential': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'payment_type': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'product_category_name_english': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'order_status': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'customer_zip_code_prefix': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'customer_city': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'customer_state': <function pandas.core.series.Series.mode(self, dropna: 'bool' = True) -> 'Series'>,
 'shipping_limit_date': 'last',
 'review_creation_date': 'last',
 'review_answer_timestamp': 'last',
 'order_purchase_timestamp': 'last',
 'order_approved_at': 'last',
 'order_delivered_carrier_date': 'last',
 'order_delivered_customer_date': 'last',
 'or

### **Groupby unique_id**

In [142]:
df_group = df.groupby(['customer_unique_id']).agg(dict_agg)

In [143]:
type(df_group['product_category_name_english'][df_group.index == '001926cef41060fae572e2e7b30bd2a4'].values)

numpy.ndarray

In [144]:
type_o = type(df_group['product_category_name_english'][df_group.index == '001926cef41060fae572e2e7b30bd2a4'].values)

for col in mode_col:
    df_group[col] = df_group[col].apply(lambda x : x[0] if type(x) == type_o else x)

df_group.head(20)

,payment_sequential,payment_type,product_category_name_english,order_status,customer_zip_code_prefix,customer_city,customer_state,shipping_limit_date,review_creation_date,review_answer_timestamp,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,response_time,price,freight_value,payment_value,payment_installments,review_score,price_total,total_order,rfm,nb_char_comment
customer_unique_id,,,,,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,1,credit_card,Home,delivered,7787,cajamar,SP,1526382678000000000,1526515200000000000,1526910288000000000,1525949787000000000,1525950678000000000,1526113080000000000,1526503717000000000,1526860800000000000,395088000000000,129.90,12.00,141.90,8.0,5.0,129.90,1.0,16.0,126
0000b849f77a49e4a4ce2b2a4ca5be3f,1,credit_card,Health_Beauty,delivered,6053,osasco,SP,1526061393000000000,1525996800000000000,1526077765000000000,1525691487000000000,1525717544000000000,1525868280000000000,1525975362000000000,1526342400000000000,80965000000000,18.90,8.29,27.19,1.0,4.0,18.90,1.0,13.0,0
0000f46a3911fa3c0805444483337064,1,credit_card,Office_Supplies,delivered,88115,sao jose,SC,1489611903000000000,1491436800000000000,1491527984000000000,1489179903000000000,1489179903000000000,1489409910000000000,1491403127000000000,1491523200000000000,91184000000000,69.00,17.22,86.22,8.0,3.0,69.00,1.0,1.0,0
0000f6ccb0745a6a4b88665a16c9f078,1,credit_card,Technology,delivered,66812,belem,PA,1508363357000000000,1509580800000000000,1509668271000000000,1507840181000000000,1507841357000000000,1507925299000000000,1509571385000000000,1510531200000000000,87471000000000,25.99,17.63,43.62,4.0,4.0,25.99,1.0,7.0,12
0004aac84e0df4da2b147fca70cf8255,1,credit_card,Technology,delivered,18040,sorocaba,SP,1511381212000000000,1511827200000000000,1512253775000000000,1510688742000000000,1510690012000000000,1510861930000000000,1511824136000000000,1512432000000000000,426575000000000,180.00,16.89,196.89,6.0,5.0,180.00,1.0,10.0,0
0004bd2a26a76fe21f786e4fbd80607f,1,credit_card,Construction_Tools,delivered,5036,sao paulo,SP,1523476139000000000,1523145600000000000,1523391212000000000,1522956796000000000,1522957739000000000,1523061532000000000,1523117563000000000,1524096000000000000,245612000000000,154.00,12.98,166.98,8.0,4.0,154.00,1.0,16.0,0
00050ab1314c0e55a6ca13cf7181fecf,1,boleto,Technology,delivered,13084,campinas,SP,1525233073000000000,1524873600000000000,1525207311000000000,1524229043000000000,1524628273000000000,1524669900000000000,1524830939000000000,1525824000000000000,333711000000000,27.99,7.39,35.38,1.0,4.0,27.99,1.0,14.0,0
00053a61a98854899e70ed204dd4bafe,1,credit_card,Sports_Leisure,delivered,80410,curitiba,PR,1520335857000000000,1521244800000000000,1521328973000000000,1519816541000000000,1519817562000000000,1519934970000000000,1521226971000000000,1522022400000000000,84173000000000,191.00,18.59,419.18,3.0,1.0,382.00,2.0,16.0,396
0005e1862207bf6ccc02e4228effd9a0,1,credit_card,Fashion_Accessories,delivered,25966,teresopolis,RJ,1489103006000000000,1489104000000000000,1489635959000000000,1488670332000000000,1488671006000000000,1488777247000000000,1489048388000000000,1491436800000000000,531959000000000,135.00,15.12,150.12,3.0,4.0,135.00,1.0,3.0,0


In [145]:
df_group = df_group.reset_index()

### **Base Argent**

In [146]:
df_group.to_sql(name='Argent', con=engine_argent, if_exists = 'append', index=False)

93394